In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [15]:
# 데이터 불러오기
match_info = pd.read_csv('data/match_info.csv')
raw_data = pd.read_csv('data/raw_data.csv')

# match_info.head()
# raw_data.head()

In [16]:
# 선수 기본 통계
player_data = raw_data[raw_data['player_id'].notna()].copy()
player_games = player_data.groupby('player_name_ko')['game_id'].nunique().to_dict()

# player_data.head()
# player_games

player_info = player_data.groupby('player_name_ko').agg({
    'team_name_ko': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'main_position': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
}).reset_index()

# print(f"총 {len(player_info)}명의 선수 데이터 확인")
# print(f"\n출전 경기 수 분포:")
# print(pd.Series(player_games).describe())

# 최소 출전 경기 수 설정 (5경기 이상)
min_games = 5
qualified_players = [p for p, g in player_games.items() if g >= min_games]

# print(f"\n{min_games}경기 이상 출전 선수: {len(qualified_players)}명")


In [17]:
# 공격 부문 통합 지표 계산

attack_awards = []

for player in qualified_players:
    p_data = player_data[player_data['player_name_ko'] == player]
    games = player_games[player]
    
    team = player_info[player_info['player_name_ko'] == player]['team_name_ko'].iloc[0]
    position = player_info[player_info['player_name_ko'] == player]['main_position'].iloc[0]
    
    # 1. 대포알 상: 슛은 많은데 Off Target이 많은 선수
    shots = p_data[p_data['type_name'].isin(['Shot', 'Shot_Freekick'])]
    total_shots = len(shots)
    off_target_shots = (shots['result_name'] == 'Off Target').sum()
    
    # 2. 새가슴 상: 패널티 박스 안에서 슛 실패
    penalty_box_shots = shots[(shots['start_x'] >= 88.5) & 
                              (shots['start_y'] >= 13.84) & 
                              (shots['start_y'] <= 54.16)]
    penalty_box_total = len(penalty_box_shots)
    penalty_box_miss = penalty_box_shots[
        ~penalty_box_shots['result_name'].isin(['Goal', 'On Target'])
    ].shape[0]
    
    # 3. 선넘네 상: 오프사이드
    offsides = len(p_data[p_data['type_name'] == 'Offside'])
    
    # 4. 내로남불 상: Pass Received는 많은데 본인 Pass는 적음
    pass_received = len(p_data[p_data['type_name'] == 'Pass Received'])
    pass_given = len(p_data[p_data['type_name'] == 'Pass'])
    receive_to_give_ratio = (pass_received / pass_given) if pass_given > 0 else 0
    
    # 5. 어디에 줘 상: Cross는 많은데 성공률 낮음
    crosses = p_data[p_data['type_name'] == 'Cross']
    total_crosses = len(crosses)
    cross_success = (crosses['result_name'] == 'Successful').sum()
    cross_fail = total_crosses - cross_success
    cross_fail_rate = (cross_fail / total_crosses * 100) if total_crosses > 0 else 0
    
    # 6. 지는 게 일상 상: Duel 패배가 많은 선수
    duels = p_data[p_data['type_name'] == 'Duel']
    total_duels = len(duels)
    duel_success = (duels['result_name'] == 'Successful').sum()
    duel_fail = total_duels - duel_success
    duel_fail_rate = (duel_fail / total_duels * 100) if total_duels > 0 else 0
    
    # 7. 키컸으면 상: 공중볼 경합 실패
    aerial_fails = (duels['result_name'] == 'Unsuccessful').sum()
    
    attack_awards.append({
        '선수명': player,
        '팀': team,
        '포지션': position,
        '경기수': games,
        # 대포알 상
        '총_슈팅': total_shots,
        '빗나간_슈팅': off_target_shots,
        '경기당_빗나간_슈팅': round(off_target_shots / games, 2),
        '슈팅_빗나감률': round((off_target_shots / total_shots * 100) if total_shots > 0 else 0, 2),
        # 새가슴 상
        '박스안_슈팅': penalty_box_total,
        '박스안_실패': penalty_box_miss,
        '경기당_박스안_실패': round(penalty_box_miss / games, 2),
        '박스안_실패율': round((penalty_box_miss / penalty_box_total * 100) if penalty_box_total > 0 else 0, 2),
        # 선넘네 상
        '오프사이드': offsides,
        '경기당_오프사이드': round(offsides / games, 2),
        # 내로남불 상
        '받은_패스': pass_received,
        '준_패스': pass_given,
        '받기vs주기_비율': round(receive_to_give_ratio, 2),
        '경기당_받은패스': round(pass_received / games, 2),
        '경기당_준패스': round(pass_given / games, 2),
        # 어디에 줘 상
        '총_크로스': total_crosses,
        '실패_크로스': cross_fail,
        '경기당_실패크로스': round(cross_fail / games, 2),
        '크로스_실패율': round(cross_fail_rate, 2),
        # 지는 게 일상 상
        '총_듀얼': total_duels,
        '패배_듀얼': duel_fail,
        '경기당_듀얼패배': round(duel_fail / games, 2),
        '듀얼_패배율': round(duel_fail_rate, 2),
        # 키컸으면 상
        '공중볼_실패': aerial_fails,
        '경기당_공중볼실패': round(aerial_fails / games, 2),
    })

attack_df = pd.DataFrame(attack_awards)


In [18]:
# 결과 
# 1. 대포알 상
print("\n대포알 상")
print("-"*80)
cannon_candidates = attack_df[attack_df['총_슈팅'] >= 10].nlargest(5, '경기당_빗나간_슈팅')
print(cannon_candidates[['선수명', '팀', '포지션', '총_슈팅', '빗나간_슈팅', '경기당_빗나간_슈팅', '슈팅_빗나감률']].to_string(index=False))

# 2. 새가슴 상
print("\n\n새가슴 상")
print("-"*80)
chicken_candidates = attack_df[attack_df['박스안_슈팅'] >= 5].nlargest(5, '경기당_박스안_실패')
print(chicken_candidates[['선수명', '팀', '포지션', '박스안_슈팅', '박스안_실패', '경기당_박스안_실패', '박스안_실패율']].to_string(index=False))

# 3. 선넘네 상
print("\n\n선넘네 상")
print("-"*80)
offside_candidates = attack_df[attack_df['오프사이드'] > 0].nlargest(5, '경기당_오프사이드')
print(offside_candidates[['선수명', '팀', '포지션', '오프사이드', '경기당_오프사이드']].to_string(index=False))

# 4. 내로남불 상
print("\n\n내로남불 상")
print("-"*80)
selfish_candidates = attack_df[
    (attack_df['받은_패스'] >= 50) & (attack_df['준_패스'] >= 10)
].nlargest(5, '받기vs주기_비율')
print(selfish_candidates[['선수명', '팀', '포지션', '받은_패스', '준_패스', '받기vs주기_비율', '경기당_받은패스', '경기당_준패스']].to_string(index=False))

# 5. 어디에 줘 상
print("\n\n어디에 줘 상")
print("-"*80)
cross_fail_candidates = attack_df[attack_df['총_크로스'] >= 10].nlargest(5, '경기당_실패크로스')
print(cross_fail_candidates[['선수명', '팀', '포지션', '총_크로스', '실패_크로스', '경기당_실패크로스', '크로스_실패율']].to_string(index=False))

# 6. 지는 게 일상 상
print("\n\n지는 게 일상 상")
print("-"*80)
duel_loser_candidates = attack_df[attack_df['총_듀얼'] >= 20].nlargest(5, '경기당_듀얼패배')
print(duel_loser_candidates[['선수명', '팀', '포지션', '총_듀얼', '패배_듀얼', '경기당_듀얼패배', '듀얼_패배율']].to_string(index=False))

# 7. 키 컸으면 상
print("\n\n키 컸으면 상: ")
print("-"*80)
aerial_fail_candidates = attack_df[attack_df['공중볼_실패'] > 0].nlargest(5, '경기당_공중볼실패')
print(aerial_fail_candidates[['선수명', '팀', '포지션', '총_듀얼', '공중볼_실패', '경기당_공중볼실패']].to_string(index=False))

print("\n" + "="*80)



대포알 상
--------------------------------------------------------------------------------
선수명    팀 포지션  총_슈팅  빗나간_슈팅  경기당_빗나간_슈팅  슈팅_빗나감률
세징야 대구FC  CF   105      45        1.61    42.86
엄지성 광주FC  LW    35      19        1.27    54.29
아사니 광주FC  RM    27      12        1.20    44.44
루카스 FC서울  LW    19       8        1.14    42.11
 야고 강원FC  CF    71      31        1.11    43.66


새가슴 상
--------------------------------------------------------------------------------
   선수명           팀 포지션  박스안_슈팅  박스안_실패  경기당_박스안_실패  박스안_실패율
유리 조나탄     제주SK FC  CF      22      14        0.58    63.64
   무고사    인천 유나이티드  CF      26      18        0.55    69.23
    야고        강원FC  CF      26      15        0.54    57.69
   정치인 김천 상무 프로축구단  LW      23      13        0.54    56.52
   티아고   전북 현대 모터스  CF      24      15        0.52    62.50


선넘네 상
--------------------------------------------------------------------------------
  선수명           팀 포지션  오프사이드  경기당_오프사이드
  이영준 김천 상무 프로축구단  CF      5       0.62
   야고 